# Prepocesamiento
1. Cargar data

In [99]:
import os

In [100]:
#Path de los datos 
data_path = r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\test_txt' 

In [101]:
#Crear un vector de documentos
#Abrimos cada archivo y lo leemos
documents = []
for filename in os.listdir(data_path):
    if filename.endswith('.txt'):
        path = os.path.join(data_path, filename)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
            documents.append((filename, content))
len(documents)

3019

 2. StopWords y Stemming

In [102]:
from nltk.stem import SnowballStemmer 

In [103]:
#Stemmer se usa para reducir las palabras a su raíz
stemmer = SnowballStemmer('english')

In [104]:
#Path de las stopwords
stopwords_path = r"C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\stopwords.txt"

In [105]:
#Abrimos el archivo y leemos las stopwords
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())

3. Tokenización

In [106]:
#Función para tokenizar y stemmizar
#tokenizar: dividir el texto en palabras
import nltk
import string

In [107]:
#Definimos una función que normaliza el texto con todos los requisitos necesarios:
#conviertimos en minúsculas
#eliminamos los signos de puntuación
#tokenizamos
#aplicamos stemming
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    processed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(processed_tokens)


4. Aplicación a todos los documentos

In [108]:
#Preprocesamos los documentos
preprocessed_documents = [(filename, preprocess_text(content)) for filename, content in documents]

In [109]:
len(preprocessed_documents)

3019

Transformacion de tupla a lista
tupla = ()
lista = []

In [110]:
folder_path = 'C:/Users/usuario/Fer-Pc/Escritorio/EPN/2024-A/SEPTIMO_SEMESTRE/RECUPERACION_DE_INFORMACION/repoMantillaRI/ProyectoRI/data/cats.txt'


In [111]:
lines = []
with open(folder_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [112]:
def build_inverted_index(lines):
    index = {}
    for line in lines:
        # Separar la ruta del documento y los términos
        parts = line.strip().split()
        document = parts[0]
        terms = parts[1:]
        
        for term in terms:
            if term in index:
                index[term].append(document)
            else:
                index[term] = [document]
    return index

# Construir el índice invertido
inverted_index = build_inverted_index(lines)

In [113]:
preprocessed_documents_list = [doc[0] if isinstance(doc, tuple) else doc for doc in preprocessed_documents]

In [114]:

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [115]:
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(preprocessed_documents_list)

In [116]:
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(preprocessed_documents_list)

In [117]:
# Construir relevant_documents_bow a partir del índice invertido y las consultas
relevant_documents_bow = {}
for query, relevant_terms in inverted_index.items():
    relevant_documents = set()
    for term in relevant_terms:
        if term in inverted_index:  # Verificar si el término está presente en el índice invertido
            relevant_documents.update(inverted_index[term])
    relevant_documents_bow[query] = list(relevant_documents)


In [121]:
# Calcular la precisión y el recall de los primeros 10 documentos recuperados
def calculate_precision_recall(inverted_index, relevant_documents_bow, documents_list, top_n=10):
    precision_sum = 0
    recall_sum = 0
    num_queries = 0
    
    for query, relevant_documents in relevant_documents_bow.items():
        retrieved_documents = set()
        
        for term in query.split():
            if term in inverted_index:
                retrieved_documents.update(inverted_index[term])
        
        # Calcular la intersección de los documentos relevantes y los recuperados
        intersection = set(retrieved_documents).intersection(relevant_documents)
        
        # Calcular la precisión y el recall
        precision = len(intersection) / top_n
        if len(relevant_documents) == 0:
            recall = 0
        else: 
            recall = len(intersection) / len(relevant_documents)
        
        precision_sum += precision
        recall_sum += recall
        num_queries += 1
    
    # Calcular el promedio de precisión y recall
    precision_avg = precision_sum / num_queries
    recall_avg = recall_sum / num_queries
    
    return precision_avg, recall_avg

# Calcular la precisión y el recall para los primeros 10 documentos recuperados
precision, recall = calculate_precision_recall(inverted_index, relevant_documents_bow, preprocessed_documents_list)

print("Precisión de los primeros 10 documentos recuperados:", precision)
print("Recall de los primeros 10 documentos recuperados:", recall)


Precisión de los primeros 10 documentos recuperados: 0.0
Recall de los primeros 10 documentos recuperados: 0.0
